# PART 0 - DATA EXPLORATION

In [ ]:
import numpy as np
import pandas as pd
import keras
import shutil
import matplotlib.pyplot as plt
import tqdm
import csv
import librosa
%matplotlib tk

In [2]:
train_labels = pd.read_csv('train.csv')
train_labels

fname                  label  manually_verified
0     00044347.wav                 Hi-hat                  0
1     001ca53d.wav              Saxophone                  1
2     002d256b.wav                Trumpet                  0
3     0033e230.wav           Glockenspiel                  1
4     00353774.wav                  Cello                  1
5     003b91e8.wav                  Cello                  0
6     003da8e5.wav                  Knock                  1
7     0048fd00.wav     Gunshot_or_gunfire                  1
8     004ad66f.wav               Clarinet                  0
9     0063ab88.wav      Computer_keyboard                  0
10    006f2f32.wav                 Hi-hat                  1
11    0075d39c.wav          Keys_jangling                  0
12    00780200.wav             Snare_drum                  0
13    0079d310.wav                Writing                  0
14    0091fc7f.wav                  Cello                  1
15    0097160c.wav               Laughter                  1
16    00ad7068.wav               Laughter                  0
17    00c5808a.wav                Tearing                  0
18    00c82919.wav                   Fart                  0
19    00c934d7.wav               Laughter                  1
20    00c9e799.wav                   Oboe                  0
21    00cb787c.wav                  Flute                  1
22    00ce569f.wav                 Hi-hat                  0
23    00d1fe46.wav                  Cough                  0
24    00d3bba3.wav              Telephone                  1
25    00d40fa2.wav             Snare_drum                  0
26    00d9fa61.wav                  Flute                  0
27    00e2b4cd.wav                   Bark                  1
28    00f88dc5.wav           Glockenspiel                  1
29    00fbb28b.wav                  Chime                  0
...            ...                    ...                ...
9443  ff71bf59.wav               Laughter                  0
9444  ff752a0c.wav               Clarinet                  1
9445  ff758ee0.wav     Gunshot_or_gunfire                  1
9446  ff7e6638.wav          Keys_jangling                  0
9447  ff8415ee.wav                  Chime                  0
9448  ff875923.wav                  Cello                  0
9449  ff8e95db.wav              Fireworks                  0
9450  ff95ac47.wav        Finger_snapping                  1
9451  ff9c6c3f.wav                Trumpet                  0
9452  ff9eae6f.wav                   Gong                  1
9453  ffa4a506.wav                  Cough                  1
9454  ffa61f00.wav                Tearing                  1
9455  ffb4c87e.wav             Tambourine                  0
9456  ffbba18f.wav               Laughter                  0
9457  ffc8d3ae.wav               Laughter                  0
9458  ffc92b01.wav                  Cello                  1
9459  ffcb5c56.wav      Computer_keyboard                  1
9460  ffcefb78.wav                  Knock                  1
9461  ffd52400.wav         Microwave_oven                  1
9462  ffd6b26c.wav                   Fart                  1
9463  ffda1d2b.wav          Keys_jangling                  1
9464  ffdf1684.wav                   Gong                  0
9465  ffe16cef.wav                  Knock                  0
9466  ffe66841.wav  Burping_or_eructation                  0
9467  ffec1b59.wav   Drawer_open_or_close                  0
9468  ffec59fb.wav              Fireworks                  0
9469  fff37590.wav                 Hi-hat                  0
9470  fff44ac6.wav               Laughter                  0
9471  fff6a13d.wav                  Chime                  0
9472  fff81f55.wav                  Cough                  1

[9473 rows x 3 columns]

In [3]:
'''category_group = train_labels.groupby(['label', 'manually_verified']).count()

plot = category_group.unstack().reindex(category_group.unstack().sum(axis=1).sort_values().index)\
          .plot(kind='bar', stacked=True, title="Number of Audio Samples per Category", figsize=(16,10))
    
plot.set_xlabel("Category")

plot.set_ylabel("Number of Samples");'''


'category_group = train_labels.groupby([\'label\', \'manually_verified\']).count()\n\nplot = category_group.unstack().reindex(category_group.unstack().sum(axis=1).sort_values().index)          .plot(kind=\'bar\', stacked=True, title="Number of Audio Samples per Category", figsize=(16,10))\n    \nplot.set_xlabel("Category")\n\nplot.set_ylabel("Number of Samples");'

<img src = "pic1.png">

In [4]:

import IPython.display as ipd  # To play sound in the notebook

fname = 'train_audio/audio_train/' + '9c25e6c0.wav'  # Hi-hat

ipd.Audio(fname)


In [5]:
import wave

wav = wave.open(fname)

print("Sampling (frame) rate = ", wav.getframerate())
print("Total samples (frames) = ", wav.getnframes())
print("Duration = ", wav.getnframes()/wav.getframerate(), "Seconds")

Sampling (frame) rate =  44100
Total samples (frames) =  454230
Duration =  10.3 Seconds


In [6]:
import librosa
import librosa.display

y, sr = librosa.load(fname, duration=8)
ps = librosa.feature.melspectrogram(y=y, sr=sr)
ps.shape
#librosa.display.specshow(ps, y_axis='mel', x_axis='time')
#plt.show()

(128, 345)

<img src = "pic2.png">

In [7]:
fname2 = 'train_audio/audio_train/' + '67841fc1.wav' 

y, sr = librosa.load(fname2, duration=8)
ps = librosa.feature.melspectrogram(y=y, sr=sr)
ps.shape
#librosa.display.specshow(ps, y_axis='mel', x_axis='time')
#plt.show()

(128, 345)

<img src = "pic3.png">

In [8]:
import glob

train_files = glob.glob("train_audio/audio_train/*.wav")
test_files = glob.glob("test_audio/audio_test/*.wav")

train_files.sort()
test_files.sort()


In [9]:
train_files[0]
len(train_files)

9473

In [10]:
n_classes = list(train_labels.label.unique())
print(n_classes)
print("**********************************")
print("number of classes = ", len(n_classes))

['Hi-hat', 'Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Knock', 'Gunshot_or_gunfire', 'Clarinet', 'Computer_keyboard', 'Keys_jangling', 'Snare_drum', 'Writing', 'Laughter', 'Tearing', 'Fart', 'Oboe', 'Flute', 'Cough', 'Telephone', 'Bark', 'Chime', 'Bass_drum', 'Bus', 'Squeak', 'Scissors', 'Harmonica', 'Gong', 'Microwave_oven', 'Burping_or_eructation', 'Double_bass', 'Shatter', 'Fireworks', 'Tambourine', 'Cowbell', 'Electric_piano', 'Meow', 'Drawer_open_or_close', 'Applause', 'Acoustic_guitar', 'Violin_or_fiddle', 'Finger_snapping']
**********************************
number of classes =  41


In [11]:
labels_train = []
with open('train_csv.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        labels_train.append(row[0])
            
print(labels_train)

['Hi-hat', 'Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Cello', 'Knock', 'Gunshot_or_gunfire', 'Clarinet', 'Computer_keyboard', 'Hi-hat', 'Keys_jangling', 'Snare_drum', 'Writing', 'Cello', 'Laughter', 'Laughter', 'Tearing', 'Fart', 'Laughter', 'Oboe', 'Flute', 'Hi-hat', 'Cough', 'Telephone', 'Snare_drum', 'Flute', 'Bark', 'Glockenspiel', 'Chime', 'Bass_drum', 'Fart', 'Cello', 'Bus', 'Squeak', 'Scissors', 'Harmonica', 'Squeak', 'Clarinet', 'Scissors', 'Clarinet', 'Gong', 'Writing', 'Microwave_oven', 'Burping_or_eructation', 'Harmonica', 'Cello', 'Double_bass', 'Double_bass', 'Shatter', 'Hi-hat', 'Fireworks', 'Hi-hat', 'Fireworks', 'Bark', 'Tambourine', 'Shatter', 'Gong', 'Harmonica', 'Double_bass', 'Telephone', 'Bass_drum', 'Writing', 'Harmonica', 'Shatter', 'Harmonica', 'Tearing', 'Microwave_oven', 'Gunshot_or_gunfire', 'Oboe', 'Bark', 'Fart', 'Keys_jangling', 'Squeak', 'Burping_or_eructation', 'Bus', 'Knock', 'Double_bass', 'Computer_keyboard', 'Cowbell', 'Microwave_oven', 'Firew

In [12]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer().fit(labels_train)
label = lb.transform(labels_train)


# PART 1 - MFCC WITHOUT DATA AUGMENTATION

In [13]:
SAMPLE_RATE = 44100

def get_mfcc(path):
    b, _ = librosa.core.load(path, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        gmm = librosa.feature.mfcc(b, sr = SAMPLE_RATE, n_mfcc=20)
        return pd.Series(np.hstack((np.mean(gmm, axis=1), np.std(gmm, axis=1))))
    except:
        print('bad file')
        return pd.Series([0]*40)

In [14]:
training_files = []

number_run_train = 0

for x in train_files:
    data = get_mfcc(x)
    training_files.append(data)
    number_run_train = number_run_train + 1
    if(number_run_train % 500 == 0):
        print(x, number_run_train)



train_audio/audio_train\0e630318.wav 500
train_audio/audio_train\1b8de5c7.wav 1000
train_audio/audio_train\28f5dfd6.wav 1500
train_audio/audio_train\372340eb.wav 2000
train_audio/audio_train\44645349.wav 2500
train_audio/audio_train\50fc3ef5.wav 3000
train_audio/audio_train\5e0d9b18.wav 3500
train_audio/audio_train\6b90ade1.wav 4000
train_audio/audio_train\7a002442.wav 4500
train_audio/audio_train\86f881f3.wav 5000
train_audio/audio_train\95b961c8.wav 5500
train_audio/audio_train\a37e6f07.wav 6000
train_audio/audio_train\b1a843d3.wav 6500
train_audio/audio_train\beddbf08.wav 7000
train_audio/audio_train\cbe40a05.wav 7500
train_audio/audio_train\d8dee93f.wav 8000
train_audio/audio_train\e61c41dc.wav 8500
train_audio/audio_train\f2f466aa.wav 9000


In [15]:
import wave
import contextlib

for a in test_files:
    with contextlib.closing(wave.open(a,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        if (duration == 0.00):
            print(a)
            test_files.remove(a)

test_audio/audio_test\0b0427e2.wav
test_audio/audio_test\6ea0099f.wav
test_audio/audio_test\b39975f5.wav


In [16]:

testing_files = []
number_run_test = 0

for y in test_files:
    file = get_mfcc(y)
    testing_files.append(file)
    number_run_test = number_run_test + 1
    if(number_run_test % 500 == 0):
        print(x, number_run_test)



train_audio/audio_train\fff81f55.wav 500
train_audio/audio_train\fff81f55.wav 1000
train_audio/audio_train\fff81f55.wav 1500
train_audio/audio_train\fff81f55.wav 2000
train_audio/audio_train\fff81f55.wav 2500
train_audio/audio_train\fff81f55.wav 3000
train_audio/audio_train\fff81f55.wav 3500
train_audio/audio_train\fff81f55.wav 4000
train_audio/audio_train\fff81f55.wav 4500
train_audio/audio_train\fff81f55.wav 5000
train_audio/audio_train\fff81f55.wav 5500
train_audio/audio_train\fff81f55.wav 6000
train_audio/audio_train\fff81f55.wav 6500
train_audio/audio_train\fff81f55.wav 7000
train_audio/audio_train\fff81f55.wav 7500
train_audio/audio_train\fff81f55.wav 8000
train_audio/audio_train\fff81f55.wav 8500
train_audio/audio_train\fff81f55.wav 9000


In [17]:
print(np.array(training_files).shape)
print(np.array(testing_files).shape)

(9473, 40)
(9397, 40)


In [18]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(training_files, label, test_size = 0.3, random_state = 2)

C:\Users\Aditya Pratap\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Reshape for CNN

In [168]:
x_train = np.array(X_train)

Xtrain = x_train.reshape(x_train.shape[0], 40, 1)

print(Xtrain.shape)

x_test = np.array(X_test)

Xtest = x_test.reshape(x_test.shape[0], 40, 1)

Xtest.shape

print(Y_train.shape)


(6631, 40, 1)
(6631, 41)


In [20]:
from keras.layers import Activation, Dense, Dropout, Conv1D, \
                         Flatten, MaxPooling1D
from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers import BatchNormalization,LSTM


In [25]:
model = Sequential()

model.add(Conv1D(64, kernel_size=(2), activation='relu', input_shape=( 40, 1)))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(64, kernel_size=(2), activation='relu'))
model.add(MaxPooling1D(pool_size=(3)))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))


model.add(Conv1D(128, kernel_size=(2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(128, kernel_size=(2), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))

model.add(Conv1D(256, kernel_size=(2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(256, kernel_size=(2), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))

model.add(LSTM(256))


model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))



model.add(Dense(41))
model.add(Activation('softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 39, 64)            192       
_________________________________________________________________
batch_normalization_8 (Batch (None, 39, 64)            156       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 38, 64)            8256      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 12, 64)            0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 12, 64)            48        
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 64)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 11, 128)           16512     
__________

In [28]:

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

# learning rate reduction
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.4, 
                                            min_lr=0.00001)

early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='auto')



callbacks_list = [learning_rate_reduction]



# fit model
hist = model.fit(Xtrain, Y_train, batch_size=100, epochs=99, validation_data = (Xtest, Y_test), callbacks = callbacks_list)



Train on 6631 samples, validate on 2842 samples
Epoch 1/99
6631/6631 [==============================] - ETA: 12s - loss: 3.2858 - acc: 0.20 - ETA: 12s - loss: 3.3339 - acc: 0.17 - ETA: 12s - loss: 3.3685 - acc: 0.19 - ETA: 12s - loss: 3.3571 - acc: 0.18 - ETA: 11s - loss: 3.3377 - acc: 0.19 - ETA: 11s - loss: 3.3278 - acc: 0.19 - ETA: 11s - loss: 3.3131 - acc: 0.19 - ETA: 11s - loss: 3.3532 - acc: 0.18 - ETA: 11s - loss: 3.3426 - acc: 0.18 - ETA: 10s - loss: 3.3325 - acc: 0.18 - ETA: 10s - loss: 3.3219 - acc: 0.18 - ETA: 10s - loss: 3.3217 - acc: 0.18 - ETA: 10s - loss: 3.3050 - acc: 0.18 - ETA: 10s - loss: 3.2828 - acc: 0.18 - ETA: 9s - loss: 3.2786 - acc: 0.1860 - ETA: 9s - loss: 3.2762 - acc: 0.184 - ETA: 9s - loss: 3.2754 - acc: 0.182 - ETA: 9s - loss: 3.2627 - acc: 0.185 - ETA: 9s - loss: 3.2512 - acc: 0.189 - ETA: 8s - loss: 3.2639 - acc: 0.188 - ETA: 8s - loss: 3.2485 - acc: 0.192 - ETA: 8s - loss: 3.2480 - acc: 0.192 - ETA: 8s - loss: 3.2450 - acc: 0.192 - ETA: 8s - loss: 3.235

6631/6631 [==============================] - ETA: 13s - loss: 1.8524 - acc: 0.45 - ETA: 13s - loss: 1.8275 - acc: 0.45 - ETA: 13s - loss: 1.9050 - acc: 0.45 - ETA: 13s - loss: 1.9262 - acc: 0.45 - ETA: 13s - loss: 1.9587 - acc: 0.44 - ETA: 12s - loss: 1.9697 - acc: 0.44 - ETA: 12s - loss: 1.9590 - acc: 0.44 - ETA: 12s - loss: 1.9440 - acc: 0.45 - ETA: 12s - loss: 1.9289 - acc: 0.45 - ETA: 11s - loss: 1.9282 - acc: 0.45 - ETA: 11s - loss: 1.9148 - acc: 0.45 - ETA: 11s - loss: 1.9107 - acc: 0.45 - ETA: 11s - loss: 1.8972 - acc: 0.45 - ETA: 11s - loss: 1.8871 - acc: 0.46 - ETA: 10s - loss: 1.8707 - acc: 0.46 - ETA: 10s - loss: 1.8801 - acc: 0.46 - ETA: 10s - loss: 1.8693 - acc: 0.46 - ETA: 10s - loss: 1.8596 - acc: 0.47 - ETA: 10s - loss: 1.8624 - acc: 0.47 - ETA: 9s - loss: 1.8807 - acc: 0.4695 - ETA: 9s - loss: 1.8868 - acc: 0.468 - ETA: 9s - loss: 1.8888 - acc: 0.467 - ETA: 9s - loss: 1.8891 - acc: 0.467 - ETA: 8s - loss: 1.8803 - acc: 0.470 - ETA: 8s - loss: 1.8747 - acc: 0.474 - ETA:

6631/6631 [==============================] - ETA: 13s - loss: 1.4027 - acc: 0.56 - ETA: 13s - loss: 1.3478 - acc: 0.58 - ETA: 13s - loss: 1.3397 - acc: 0.59 - ETA: 13s - loss: 1.3582 - acc: 0.57 - ETA: 13s - loss: 1.3506 - acc: 0.59 - ETA: 12s - loss: 1.3737 - acc: 0.58 - ETA: 12s - loss: 1.4006 - acc: 0.57 - ETA: 12s - loss: 1.3978 - acc: 0.57 - ETA: 12s - loss: 1.4543 - acc: 0.56 - ETA: 12s - loss: 1.4537 - acc: 0.56 - ETA: 11s - loss: 1.4463 - acc: 0.56 - ETA: 11s - loss: 1.4351 - acc: 0.57 - ETA: 11s - loss: 1.4316 - acc: 0.57 - ETA: 11s - loss: 1.4425 - acc: 0.56 - ETA: 10s - loss: 1.4490 - acc: 0.56 - ETA: 10s - loss: 1.4415 - acc: 0.57 - ETA: 10s - loss: 1.4316 - acc: 0.57 - ETA: 10s - loss: 1.4278 - acc: 0.57 - ETA: 10s - loss: 1.4214 - acc: 0.57 - ETA: 9s - loss: 1.4216 - acc: 0.5745 - ETA: 9s - loss: 1.4238 - acc: 0.574 - ETA: 9s - loss: 1.4245 - acc: 0.574 - ETA: 9s - loss: 1.4137 - acc: 0.579 - ETA: 9s - loss: 1.3993 - acc: 0.584 - ETA: 8s - loss: 1.4055 - acc: 0.581 - ETA:

6631/6631 [==============================] - ETA: 14s - loss: 1.2165 - acc: 0.60 - ETA: 14s - loss: 1.1627 - acc: 0.61 - ETA: 14s - loss: 1.0597 - acc: 0.66 - ETA: 13s - loss: 1.0603 - acc: 0.65 - ETA: 13s - loss: 1.0957 - acc: 0.63 - ETA: 13s - loss: 1.0797 - acc: 0.64 - ETA: 13s - loss: 1.0612 - acc: 0.65 - ETA: 12s - loss: 1.0507 - acc: 0.65 - ETA: 12s - loss: 1.0360 - acc: 0.66 - ETA: 12s - loss: 1.0185 - acc: 0.68 - ETA: 12s - loss: 1.0274 - acc: 0.67 - ETA: 11s - loss: 1.0304 - acc: 0.67 - ETA: 11s - loss: 1.0404 - acc: 0.67 - ETA: 11s - loss: 1.0487 - acc: 0.67 - ETA: 11s - loss: 1.0427 - acc: 0.67 - ETA: 11s - loss: 1.0422 - acc: 0.67 - ETA: 10s - loss: 1.0374 - acc: 0.67 - ETA: 10s - loss: 1.0302 - acc: 0.68 - ETA: 10s - loss: 1.0486 - acc: 0.67 - ETA: 10s - loss: 1.0425 - acc: 0.67 - ETA: 9s - loss: 1.0464 - acc: 0.6781 - ETA: 9s - loss: 1.0421 - acc: 0.680 - ETA: 9s - loss: 1.0410 - acc: 0.681 - ETA: 9s - loss: 1.0425 - acc: 0.682 - ETA: 9s - loss: 1.0553 - acc: 0.679 - ETA:

6631/6631 [==============================] - ETA: 14s - loss: 0.9373 - acc: 0.70 - ETA: 14s - loss: 0.8367 - acc: 0.74 - ETA: 14s - loss: 0.7937 - acc: 0.76 - ETA: 14s - loss: 0.8011 - acc: 0.76 - ETA: 13s - loss: 0.7851 - acc: 0.75 - ETA: 13s - loss: 0.8000 - acc: 0.75 - ETA: 13s - loss: 0.8407 - acc: 0.74 - ETA: 12s - loss: 0.8471 - acc: 0.73 - ETA: 12s - loss: 0.8487 - acc: 0.73 - ETA: 12s - loss: 0.8480 - acc: 0.73 - ETA: 12s - loss: 0.8696 - acc: 0.72 - ETA: 12s - loss: 0.8766 - acc: 0.72 - ETA: 11s - loss: 0.8726 - acc: 0.72 - ETA: 11s - loss: 0.8700 - acc: 0.72 - ETA: 11s - loss: 0.8733 - acc: 0.72 - ETA: 11s - loss: 0.8773 - acc: 0.72 - ETA: 10s - loss: 0.8558 - acc: 0.73 - ETA: 10s - loss: 0.8463 - acc: 0.73 - ETA: 10s - loss: 0.8420 - acc: 0.74 - ETA: 10s - loss: 0.8355 - acc: 0.74 - ETA: 10s - loss: 0.8375 - acc: 0.74 - ETA: 9s - loss: 0.8442 - acc: 0.7432 - ETA: 9s - loss: 0.8444 - acc: 0.742 - ETA: 9s - loss: 0.8489 - acc: 0.740 - ETA: 9s - loss: 0.8406 - acc: 0.743 - ETA:

...

6631/6631 [==============================] - ETA: 16s - loss: 0.2674 - acc: 0.94 - ETA: 16s - loss: 0.3070 - acc: 0.93 - ETA: 16s - loss: 0.3169 - acc: 0.92 - ETA: 16s - loss: 0.3379 - acc: 0.90 - ETA: 15s - loss: 0.3280 - acc: 0.91 - ETA: 15s - loss: 0.3278 - acc: 0.90 - ETA: 15s - loss: 0.3399 - acc: 0.90 - ETA: 15s - loss: 0.3300 - acc: 0.90 - ETA: 15s - loss: 0.3190 - acc: 0.91 - ETA: 14s - loss: 0.3274 - acc: 0.90 - ETA: 14s - loss: 0.3317 - acc: 0.90 - ETA: 14s - loss: 0.3328 - acc: 0.90 - ETA: 14s - loss: 0.3319 - acc: 0.90 - ETA: 14s - loss: 0.3222 - acc: 0.90 - ETA: 13s - loss: 0.3248 - acc: 0.90 - ETA: 13s - loss: 0.3188 - acc: 0.90 - ETA: 13s - loss: 0.3220 - acc: 0.90 - ETA: 12s - loss: 0.3269 - acc: 0.90 - ETA: 12s - loss: 0.3250 - acc: 0.90 - ETA: 12s - loss: 0.3227 - acc: 0.90 - ETA: 12s - loss: 0.3279 - acc: 0.90 - ETA: 11s - loss: 0.3217 - acc: 0.90 - ETA: 11s - loss: 0.3229 - acc: 0.90 - ETA: 11s - loss: 0.3281 - acc: 0.89 - ETA: 11s - loss: 0.3368 - acc: 0.89 - ETA: 

6631/6631 [==============================] - ETA: 16s - loss: 0.4408 - acc: 0.84 - ETA: 16s - loss: 0.4069 - acc: 0.87 - ETA: 16s - loss: 0.3791 - acc: 0.87 - ETA: 16s - loss: 0.3563 - acc: 0.88 - ETA: 15s - loss: 0.3343 - acc: 0.89 - ETA: 15s - loss: 0.3209 - acc: 0.89 - ETA: 15s - loss: 0.3237 - acc: 0.90 - ETA: 15s - loss: 0.3143 - acc: 0.90 - ETA: 14s - loss: 0.3157 - acc: 0.90 - ETA: 14s - loss: 0.3175 - acc: 0.89 - ETA: 14s - loss: 0.3226 - acc: 0.89 - ETA: 14s - loss: 0.3247 - acc: 0.89 - ETA: 13s - loss: 0.3276 - acc: 0.89 - ETA: 13s - loss: 0.3361 - acc: 0.89 - ETA: 13s - loss: 0.3376 - acc: 0.89 - ETA: 13s - loss: 0.3281 - acc: 0.89 - ETA: 13s - loss: 0.3256 - acc: 0.89 - ETA: 12s - loss: 0.3227 - acc: 0.89 - ETA: 12s - loss: 0.3251 - acc: 0.89 - ETA: 12s - loss: 0.3258 - acc: 0.89 - ETA: 11s - loss: 0.3244 - acc: 0.89 - ETA: 11s - loss: 0.3261 - acc: 0.89 - ETA: 11s - loss: 0.3335 - acc: 0.89 - ETA: 11s - loss: 0.3343 - acc: 0.89 - ETA: 10s - loss: 0.3346 - acc: 0.89 - ETA: 

6631/6631 [==============================] - ETA: 17s - loss: 0.2121 - acc: 0.93 - ETA: 17s - loss: 0.2420 - acc: 0.92 - ETA: 17s - loss: 0.2408 - acc: 0.93 - ETA: 16s - loss: 0.2769 - acc: 0.92 - ETA: 16s - loss: 0.2807 - acc: 0.91 - ETA: 15s - loss: 0.2877 - acc: 0.90 - ETA: 15s - loss: 0.2944 - acc: 0.90 - ETA: 15s - loss: 0.2994 - acc: 0.90 - ETA: 15s - loss: 0.2908 - acc: 0.90 - ETA: 15s - loss: 0.3052 - acc: 0.89 - ETA: 14s - loss: 0.3172 - acc: 0.89 - ETA: 14s - loss: 0.3390 - acc: 0.89 - ETA: 14s - loss: 0.3390 - acc: 0.89 - ETA: 13s - loss: 0.3410 - acc: 0.89 - ETA: 13s - loss: 0.3355 - acc: 0.89 - ETA: 13s - loss: 0.3367 - acc: 0.89 - ETA: 13s - loss: 0.3323 - acc: 0.90 - ETA: 12s - loss: 0.3338 - acc: 0.90 - ETA: 12s - loss: 0.3377 - acc: 0.90 - ETA: 12s - loss: 0.3329 - acc: 0.90 - ETA: 11s - loss: 0.3379 - acc: 0.89 - ETA: 11s - loss: 0.3391 - acc: 0.89 - ETA: 11s - loss: 0.3401 - acc: 0.89 - ETA: 11s - loss: 0.3412 - acc: 0.89 - ETA: 10s - loss: 0.3395 - acc: 0.89 - ETA: 

6631/6631 [==============================] - ETA: 20s - loss: 0.3585 - acc: 0.89 - ETA: 20s - loss: 0.3493 - acc: 0.89 - ETA: 19s - loss: 0.3308 - acc: 0.90 - ETA: 18s - loss: 0.3353 - acc: 0.89 - ETA: 17s - loss: 0.3588 - acc: 0.89 - ETA: 17s - loss: 0.3721 - acc: 0.89 - ETA: 17s - loss: 0.3683 - acc: 0.89 - ETA: 17s - loss: 0.3726 - acc: 0.89 - ETA: 16s - loss: 0.3654 - acc: 0.89 - ETA: 16s - loss: 0.3580 - acc: 0.89 - ETA: 16s - loss: 0.3521 - acc: 0.89 - ETA: 15s - loss: 0.3398 - acc: 0.89 - ETA: 15s - loss: 0.3333 - acc: 0.90 - ETA: 15s - loss: 0.3338 - acc: 0.90 - ETA: 15s - loss: 0.3412 - acc: 0.89 - ETA: 14s - loss: 0.3308 - acc: 0.90 - ETA: 14s - loss: 0.3304 - acc: 0.89 - ETA: 14s - loss: 0.3237 - acc: 0.90 - ETA: 14s - loss: 0.3318 - acc: 0.90 - ETA: 13s - loss: 0.3334 - acc: 0.90 - ETA: 13s - loss: 0.3267 - acc: 0.90 - ETA: 13s - loss: 0.3293 - acc: 0.90 - ETA: 12s - loss: 0.3358 - acc: 0.90 - ETA: 12s - loss: 0.3344 - acc: 0.90 - ETA: 12s - loss: 0.3449 - acc: 0.89 - ETA: 

6631/6631 [==============================] - ETA: 20s - loss: 0.2944 - acc: 0.91 - ETA: 20s - loss: 0.3221 - acc: 0.88 - ETA: 20s - loss: 0.3168 - acc: 0.88 - ETA: 19s - loss: 0.3194 - acc: 0.88 - ETA: 19s - loss: 0.3157 - acc: 0.89 - ETA: 18s - loss: 0.3679 - acc: 0.87 - ETA: 18s - loss: 0.3726 - acc: 0.87 - ETA: 18s - loss: 0.3822 - acc: 0.87 - ETA: 18s - loss: 0.3654 - acc: 0.88 - ETA: 17s - loss: 0.3750 - acc: 0.88 - ETA: 17s - loss: 0.3813 - acc: 0.87 - ETA: 17s - loss: 0.3755 - acc: 0.88 - ETA: 17s - loss: 0.3669 - acc: 0.88 - ETA: 17s - loss: 0.3643 - acc: 0.88 - ETA: 16s - loss: 0.3570 - acc: 0.88 - ETA: 16s - loss: 0.3627 - acc: 0.88 - ETA: 16s - loss: 0.3624 - acc: 0.88 - ETA: 16s - loss: 0.3654 - acc: 0.88 - ETA: 15s - loss: 0.3684 - acc: 0.88 - ETA: 15s - loss: 0.3643 - acc: 0.88 - ETA: 15s - loss: 0.3616 - acc: 0.88 - ETA: 14s - loss: 0.3555 - acc: 0.88 - ETA: 14s - loss: 0.3625 - acc: 0.88 - ETA: 14s - loss: 0.3569 - acc: 0.88 - ETA: 14s - loss: 0.3574 - acc: 0.88 - ETA: 

In [29]:
testing_files = np.array(testing_files)
testing_files = testing_files.reshape(testing_files.shape[0], 40, 1)

In [30]:
yhat = model.predict(testing_files)
test_y = lb.inverse_transform(yhat)

print(test_y)

['Tearing' 'Saxophone' 'Computer_keyboard' ..., 'Cello' 'Bass_drum'
 'Applause']


In [31]:
import os

test_labels = []
for filename in test_files:
    head, tail = os.path.split(os.path.split(filename)[1]) #Split the id and ".png" of image
    test_labels.append(tail)

In [32]:
df = pd.DataFrame(data={'fname': test_labels, 'label': test_y})
df_sort = df.sort_values(by=['fname'])
df_sort.to_csv('results.csv', index=False)

# PART 2-  MODEL WITH LIBROSA AND DATA AUGMENTATION

### 1. Vary Pitch

In [130]:
augmented_data_pitch = []
training_files_ar = []

augmented_data_pitch.clear()
training_files_ar.clear()
counter = 0

for x in train_files:    
    y, sr = librosa.load(x)
    training_files_ar.append(y)
    y_third = librosa.effects.pitch_shift(y, sr, n_steps=4)
    augmented_data_pitch.append(y_third)
    counter = counter + 1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336


8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612


### 2. Vary Time

In [ ]:
augmented_data_time = []
counter = 0

for x in train_files:    
    y, sr = librosa.load(x, duration=2.97)
    y_changed = librosa.effects.time_stretch(y, rate=0.81)
    augmented_data_time.append(y_changed)
    counter = counter + 1

In [131]:
augmented_time_labels = labels_train
augmented_pitch_labels = labels_train

#COMBINE ALL ARRAYS (TRAINING DATA) AND LABELS

In [132]:
aug_train = []
aug_labels = []

aug_train = np.concatenate((training_files_ar, augmented_data_pitch, augmented_data_time), axis = 0)
aug_labels = np.concatenate((labels_train, augmented_pitch_labels, augmented_time_labels), axis = 0)

In [154]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer().fit(aug_labels)
labels = lb.transform(aug_labels)


In [202]:
x_train_aug, x_test_aug, y_train_aug, y_test_aug = train_test_split(aug_train, labels, test_size = 0.3, random_state = 2)

In [204]:
print("X-TRAIN", np.array(x_train_aug).shape)
print("X-TEST", np.array(x_test_aug).shape)
print("Y-TRAIN", np.array(y_train_aug).shape)
print("Y-TEST", np.array(y_test_aug).shape)

X-TRAIN (19893,)
X-TEST (8526,)
Y-TRAIN (19893, 41)
Y-TEST (8526, 41)


In [197]:
x_train_aug = np.array(x_train_aug)
x_test_aug = np.array(x_test_aug)

In [187]:
from keras.layers import Activation, Dense, Dropout, Conv1D, \
                         Flatten, MaxPooling1D
from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers import BatchNormalization,LSTM

In [196]:
model = Sequential()

model.add(Conv1D(64, kernel_size=(2), activation='relu', input_shape=( 1, 1)))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(64, kernel_size=(2), activation='relu'))
model.add(MaxPooling1D(pool_size=(3)))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))


model.add(Conv1D(128, kernel_size=(2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(128, kernel_size=(2), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))

model.add(Conv1D(256, kernel_size=(2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(256, kernel_size=(2), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))

model.add(LSTM(256))


model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))



model.add(Dense(41))
model.add(Activation('softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'conv1d_57/convolution/Conv2D' (op: 'Conv2D') with input shapes: [?,1,1,1], [1,2,1,64].

In [193]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

# learning rate reduction
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.4, 
                                            min_lr=0.00001)

early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='auto')



callbacks_list = [learning_rate_reduction]



# fit model
hist = model.fit(Xtrain_aug, Ytrain_aug, batch_size=100, epochs=99, callbacks = callbacks_list)



ValueError: Error when checking input: expected conv1d_51_input to have 3 dimensions, but got array with shape (19893, 1)

In [209]:
you take the part between the first and the fifth second and reshape that?
yeah basically
it isnt a defined length so you can mess around with it and you dont reshape it tha
t 
just is the shape

okay so lets say we have an audio file and we take the first second to the fifth and reshape to 19893, something, something?

no
the whole and final matrix will look like this

number of series  || series length || 1
so you make each few seconds its own input if that makes sense

for file in audio_files:
    inputs = []
    outputs = []
    for x in range(0, length of audio - sequence length)
        indivisual input = librosa offse x - sequence length
        output = file.label
        input.append(indivisual input)
        output.append(output)
        
    librosa.load()

SyntaxError: invalid syntax (<ipython-input-209-4709e6b276d0>, line 2)